In [1]:
import cv2 as cv
import numpy as np
import os

In [2]:
def main(img_path1, img_path2):
    img1 = cv.imread(img_path1)
    img2 = cv.imread(img_path2)
    if img1 is None or img2 is None:
        raise FileNotFoundError("Could not read one of the images.")

    g1 = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
    g2 = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)

    orb = cv.ORB_create(nfeatures=4000)
    k1, d1 = orb.detectAndCompute(g1, None)
    k2, d2 = orb.detectAndCompute(g2, None)

    bf = cv.BFMatcher(cv.NORM_HAMMING)
    matches = bf.knnMatch(d1, d2, k=2)

    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append(m)

    pts1 = np.float32([k1[m.queryIdx].pt for m in good])
    pts2 = np.float32([k2[m.trainIdx].pt for m in good])

    H, mask = cv.findHomography(pts1, pts2, method=cv.RANSAC, ransacReprojThreshold=3.0)
    if H is None:
        raise RuntimeError("Homography estimation failed.")
    inliers = int(mask.sum())
    print(f"Good matches: {len(good)} | Homography inliers: {inliers}")
    print("H:\n", H)

    # Warp img1 into img2 frame
    h2, w2 = img2.shape[:2]
    warped = cv.warpPerspective(img1, H, (w2, h2))

    # Simple overlay visualization
    overlay = cv.addWeighted(img2, 0.5, warped, 0.5, 0)

    cv.imshow("Image2", img2)
    cv.imshow("Warped Image1 -> Image2 via H", warped)
    cv.imshow("Overlay (should align if planar / pure rotation)", overlay)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
img_dir = r'D:\Python things\middle-ml-cv-roadmap\data\raw'
img_1 = 'img_example_11.jpg'
img_2 = 'img_example_12.jpg'
img_1_path = os.path.join(img_dir, img_1)
img_2_path = os.path.join(img_dir, img_2)

In [4]:
main(img_1_path, img_2_path)

Good matches: 298 | Homography inliers: 130
H:
 [[ 5.42636476e-01 -3.38287870e-02  1.49160171e+02]
 [-1.80946754e-01  8.04512604e-01  1.01711475e+02]
 [-4.24107426e-04 -2.01112578e-05  1.00000000e+00]]
